In [2]:
s = None

def Oracle_Connect():
    import socket
    global s
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        s.connect(('128.8.130.16', 49101))
    except socket.error as e:
        print(e)
        return -1

    print("Connected to server successfully.")

    return 0

def Oracle_Disconnect():
    if not s:
        print("[WARNING]: You haven't connected to the server yet.")
        return -1

    s.close()
    print("Connection closed successfully.")

    return 0

# Packet Structure: < num_blocks(1) || ciphertext(16*num_blocks) || null-terminator(1) >
def Oracle_Send(ctext, num_blocks):
    if not s:
        print("[WARNING]: You haven't connected to the server yet.")
        return -1

    msg = ctext[:]
    msg.insert(0, num_blocks)
    msg.append(0)

    s.send(bytearray(msg))
    
    ### IMPLEMENTATION FOR NUMPY ARRAYS ###
#     msg = np.concatenate(([num_blocks], ctext, [0])).astype(np.uint8)
#     s.send(msg.tobytes())
    
    recvbit = s.recv(2)

    try:
        return int(recvbit)
    except ValueError as e:
        return int(recvbit.decode()[0])

Alrighty, let's get to cracking.
Firstly, we take the block length and ciphertext as given in the assignment:

In [3]:
L = 16

# The ciphertext
data = "9F0B13944841A832B2421B9EAF6D9836813EC9D944A5C8347A7CA69AA34D8DC0DF70E343C4000A2AE35874CE75E64C31"

We're implementing a padding oracle attack.
For the concept of the attack, see $\href{https://github.com/Noam-Coh3n/ModCrypto/blob/main/POA.pdf}{\text{these slides}}$ or $\href{https://en.wikipedia.org/wiki/Padding_oracle_attack#Attacks_using_padding_oracles}{\text{this wikipedia page}}$.
The last block has padding, so we want a function that determines the exact padding.
For convenience, we'll first define a function that flips a bit in a certain position in a ciphertext, sends it to the oracle for verification and then restores the original ciphertext.

In [ ]:
def check_flipped(cc, i):
    cc[i] ^= 1               # Change byte i
    rc = Oracle_Send(cc, 2)
    cc[i] ^= 1               # Restore byte i
    return rc

To determine the padding, we need the last two blocks of the ciphertext, $C_1$ and $C_2$, since the first one is used to xor with the output of the decryption function on the second block, $\mathrm{Dec}_k(C_2)$ to determine the plaintext $P_2 = \mathrm{Dec}_k(C_2) \oplus C_1$.
Now, starting at $C_1[0]$, we flip a bit, changing the byte, and send the new $C_1'$ to the server for verification, since this bit is xor'ed with a bit in $\mathrm{Dec}_k(C_2)$, this results in a plaintext $P_2'$ which differs from $P_2$ precisely on byte $0$.
If the server verifies the padding, we know byte $0$ wasn't part of the padding, so we flip $C_1[0]$ back and move on to $C_1[1]$.
Since there is padding of the form $\overbrace{p\;p \cdots p}^{p \text{times}}$ for some byte value $p$, this process will eventually reach a byte with value $p$ that's part of the padding and change its value.
Since $p-1$ times a $p$ at the end isn't a valid padding anymore, the server will then give an error.
If this happens on position $i$ with blocklength $L$, we know the padding has length $L-i$ and thus also have value $L-i$.

In [ ]:
def padding(cc):
    i = 0
    while check_flipped(cc, i):
        i += 1
        
    return L - i

Given the padding of $P_2$, we want to use this to obtain the rest of $P_2$, which we can do as follows:
We know $P_2 = \mathrm{Dec}_k(C_2) \oplus C_1$ ends in $p$ times a byte with value $p$, now to determine the last byte before the padding (call this position $x$), we can change the value of $C_1$ in a way, that we force the last $p$ bytes to equal $p+1$:
simply xor the last $p$ bits with $p \oplus (p+1)$, then $$P'_2 = \mathrm{Dec}_k(C_2) \oplus C'_1 = \mathrm{Dec}_k(C_2) \oplus C_1 \oplus p \oplus (p + 1) = P_2 \oplus p \oplus (p+1).$$
For the last $p$ bytes, $P_2 = p$, so this becomes
$$
    p \oplus p \oplus (p+1) = p+1.
$$
Now, the server will reject this because of invalid padding, unless the value of $P_2'$ at $x$ is equal to $p+1$ as well, since $p+1$ times the value $p+1$ is a valid padding again.
We can loop through all the possible byte values $b$ for $C_1[x]$ until the server accepts the padding, at which point we'll know $P_2'[x] = p+1$.
Then $\mathrm{Dec}_k(C_2)[x]$ must equal $b \oplus (p+1)$, so $P_2[x] = b \oplus (p+1) \oplus C_1[x]$. 
We can now let the last $p+1$ bytes be $p+2$ and repeat the process for the byte at position $x-1$.
Doing this for all the bytes will yields the original message.

In [ ]:
def decrypt(cc, p):
    m = [0]*(L-p)
    for pp in range(p + 1, L + 1):             # Let pp be the new padding value (p+1 in the explanation above)
        for pad_byte in range(L-1, L-pp, -1):  # loop through last pp-1 bytes of C_1
            cc[pad_byte] ^= pp ^ (pp - 1)      # force P'_2 to have pp as last (pp-1) values

        x   = L - pp
        C1x = cc[x]
        
        for b in range(256):
            cc[x] = b
    
            # check if padding is accepted
            if Oracle_Send(cc, 2):
                m[x] = b ^ pp ^ C1x
                break
    return m

Now, lets put it all together in one nice function:

In [12]:
def crack(data, blocks):
    Oracle_Connect()

    # convert ciphertext data into list of integers
    ctext = [(int(data[i : i + 2], 16)) for i in range(0, len(data), 2)]

    rc = Oracle_Send(ctext, blocks)
    print(f"Oracle returned: {rc}")
    
    # decryption requires two consecutive blocks
    CC = [ctext[i:i+2*L] for i in range(0, len(ctext) - L, L)]

    m = ''

    pad = True
    for cc in CC[::-1]:
        p = padding(cc) if pad else 0
        m = ''.join([chr(x) for x in decrypt(cc, p)]) + m
        
        # only last block has padding
        pad = False                             

    Oracle_Disconnect()
    return m

In [11]:
m = crack(data, 3)
print(f'The original messge was: {m}.')

Connected to server successfully.
Oracle returned: 1
Connection closed successfully.
The original messge was: Yay! You get an A. =).
